In [1]:
import requests
import tkinter as tk
from tkinter import messagebox
import webbrowser

CHEAPSHARK = "https://www.cheapshark.com/api/1.0"
DEALS_URL = CHEAPSHARK + "/deals"
STORES_URL = CHEAPSHARK + "/stores"
FX_URL = "https://api.exchangerate-api.com/v4/latest/USD"
REDIRECT = "https://www.cheapshark.com/redirect?dealID={deal_id}"

def stores():
    try:
        r = requests.get(STORES_URL, timeout=10)
        r.raise_for_status()
        data = r.json()
        return {str(s["storeID"]): s["storeName"] for s in data}
    except:
        return {}

def deals(title="", page=0, page_size=10):
    params = {"title": title, "pageSize": page_size, "pageNumber": page}
    try:
        r = requests.get(DEALS_URL, params=params, timeout=10)
        r.raise_for_status()
        return r.json()
    except:
        return None

def php():
    try:
        r = requests.get(FX_URL, timeout=10)
        r.raise_for_status()
        fx = r.json()
        return float(fx["rates"]["PHP"])
    except:
        return 1.0

def open_deal(deal_id):
    url = REDIRECT.format(deal_id=deal_id)
    webbrowser.open(url)

class CheapShark:
    def __init__(self, master):
        self.master = master
        master.title("Deal Tracker Utilizing CheapShark")

        tk.Label(master, text="Game Title:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
        self.search_entry = tk.Entry(master, width=30)
        self.search_entry.grid(row=0, column=1, padx=5, pady=5)

        self.search_button = tk.Button(master, text="Search Deals", command=self.fetch_deals)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)

        self.output_frame = tk.Frame(master)
        self.output_frame.grid(row=1, column=0, columnspan=3, padx=10, pady=10, sticky="w")

        self.pagination_frame = tk.Frame(master)
        self.pagination_frame.grid(row=2, column=0, columnspan=3, pady=10)

        self.prev_button = tk.Button(self.pagination_frame, text="Previous", command=self.prev_page)
        self.prev_button.pack(side="left", padx=5)

        self.next_button = tk.Button(self.pagination_frame, text="Next", command=self.next_page)
        self.next_button.pack(side="left", padx=5)

        self.page_label = tk.Label(self.pagination_frame, text="Page 1")
        self.page_label.pack(side="left", padx=5)

        self.stores = stores()
        self.fx_rate = php()
        self.current_page = 0
        self.page_size = 10
        self.fetch_deals()

    def fetch_deals(self):
        for w in self.output_frame.winfo_children():
            w.destroy()

        title = self.search_entry.get().strip()
        data = deals(title, page=self.current_page, page_size=self.page_size)
        if not data:
            messagebox.showerror("Error", "No deals found or API error.")
            return

        try:
            for d in data:
                store = self.stores.get(str(d.get("storeID")), "Unknown Store")
                game = d.get("title", "Unknown")
                sale = float(d.get("salePrice", 0))
                normal = float(d.get("normalPrice", 0))
                discount = float(d.get("savings", 0))
                php_price = sale * self.fx_rate
                deal_id = d.get("dealID")

                info = f"{game} — {store}\nNormal: ${normal:.2f} | Sale: ${sale:.2f} (₱{php_price:,.2f}) | Discount: {discount:.0f}%"
                tk.Label(self.output_frame, text=info, font=("Arial", 11), justify="left").pack(anchor="w", pady=5)

                link = tk.Label(self.output_frame, text="Open Deal", fg="blue", cursor="hand2", font=("Arial", 10, "underline"))
                link.pack(anchor="w")
                link.bind("<Button-1>", lambda e, deal_id=deal_id: open_deal(deal_id))

            self.page_label.config(text=f"Page {self.current_page + 1}")

        except Exception as e:
            messagebox.showerror("Error", f"Unexpected data format: {e}")

    def next_page(self):
        self.current_page += 1
        self.fetch_deals()

    def prev_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.fetch_deals()

if __name__ == "__main__":
    root = tk.Tk()
    app = CheapShark(root)
    root.mainloop()